In [1]:
import pandas as pd

class Wahlkreis:
    def __init__(self, id, name, num_votes, results):
        self.id = id
        self.name = name
        self.num_votes = num_votes
        self.results = results
        
    def print_results(self):
        print(f"Wahlkreis: {self.name}, Stimmen gesamt: {self.num_votes}")
        print("Ergebnisse:")
        for partei, percentage in self.results['relative'].items():
            print(f"  {partei}: {percentage:.2%}")

def lese_wahlkreise_aus_csv(datei_pfad):
    """
    Liest Wahlkreisdaten aus einer CSV-Datei und erstellt Wahlkreis-Objekte.
    
    :param datei_pfad: Pfad zur CSV-Datei
    :return: Liste von Wahlkreis-Objekten
    """
    # CSV-Datei einlesen
    dataframe = pd.read_csv(datei_pfad)
    
    # Parteien aus den Spalten extrahieren (alle ab der 4. Spalte)
    parteien = dataframe.columns[3:]
    
    # Liste der Wahlkreis-Objekte erstellen
    wahlkreise = []
    for _, row in dataframe.iterrows():
        nummer = int(row['id'])
        name = row['wahlkreisname']
        gesamt = int(row['gesamt'])
        
        ergebnisse = {'absolute': {}, 'relative': {}}
        for partei in parteien:
            if not pd.isna(row[partei]):
                ergebnisse['absolute'][partei] = int(row[partei])
                ergebnisse['relative'][partei] = row[partei]/gesamt
            else:
                ergebnisse['absolute'][partei] = 0
                ergebnisse['relative'][partei] = 0.0
            
        wahlkreise.append(Wahlkreis(nummer, name, gesamt, ergebnisse))
    
    return wahlkreise

def get_similarity_matrix(wahlkreise):
    matrix = {}
    
    for v in wahlkreise:
        matrix[v.id] = {}
        for w in wahlkreise:
            similarity = sum(min(v.results['relative'][p], w.results['relative'][p]) for p in v.results['relative'])
            matrix[v.id][w.id] = similarity
            
    return matrix

In [2]:
# Beispiel zur Nutzung
datei_pfad = 'bundestagswahl_2021.csv'  # Ersetzen mit dem tatsächlichen Pfad
wahlkreise = lese_wahlkreise_aus_csv(datei_pfad)

similarity_matrix = get_similarity_matrix(wahlkreise)

In [4]:
wahlkreis = 83
similarity_list = sorted([w.id for w in wahlkreise], key=lambda x: similarity_matrix[wahlkreis][x])

for w in similarity_list:
    print(w, wahlkreise[w-1].name)
    print(similarity_matrix[wahlkreis][w])

230 Rottal-Inn
0.44260426711138523
231 Straubing
0.4483660741446003
227 Deggendorf
0.45070811453225973
234 Schwandorf
0.46744722040911396
298 St. Wendel
0.4688718130640279
212 Altötting
0.4748377753371133
235 Weiden
0.4776872875290404
299 Homburg
0.4784828516911829
297 Saarlouis
0.48143661032716484
254 Donau-Ries
0.49206789092251424
228 Landshut
0.49812077192635323
239 Hof
0.5001983009518309
229 Passau
0.5007619150263425
240 Kulmbach
0.5029143329158119
223 Bad Tölz-Wolfratshausen – Miesbach
0.5053912141989364
232 Amberg
0.5092039933632767
164 Erzgebirgskreis I
0.5104165048705862
248 Bad Kissingen
0.5114760604895752
210 Pirmasens
0.511604126560774
158 Sächsische Schweiz-Osterzgebirge
0.5123850870725002
202 Bitburg
0.5128637041671865
238 Coburg
0.5134106369538222
74 Mansfeld
0.5134498040564217
225 Traunstein
0.5147454315800117
65 Elbe-Elster – Oberspreewald-Lausitz II
0.5153529222685268
73 Burgenland – Saalekreis
0.5157500331516965
296 Saarbrücken
0.515998674007347
226 Weilheim
0.5169407

In [5]:
import json

# Dateiname der JSON-Datei
datei_pfad = "landtagswahl_niedersachsen_2021_wahlkreis_vergleich.json"

# Speichern des 2D-Dictionaries
with open(datei_pfad, "w", encoding="utf-8") as file:
    json.dump(similarity_matrix, file, indent=4, ensure_ascii=False)

print(f"Die Datei wurde als {datei_pfad} gespeichert.")

Die Datei wurde als landtagswahl_niedersachsen_2021_wahlkreis_vergleich.json gespeichert.
